In [16]:
import pandas as pd
import time
import logging

from pyspark.sql import SparkSession

from soda.scan import Scan

from keys import API_KEY_ID, API_KEY_SECRET

Создание **Spark** сессии:

In [17]:
spark = SparkSession.builder.appName("Test").getOrCreate()

Читаем наш датасет в **Spark DataFrame** и задаём размер батча для имитации непрерывного потока данных как в проде:

In [18]:
df = spark.read.csv("data/Ship_Performance_Dataset.csv", header=True, inferSchema=True)
batch_size = 100  
total_rows = df.count()

В цикле нарезается один батч от датасета, указываем для **Soda** источник данных, ставим конфигурацию и загружаем проверки из файла **checks.yaml**, после этого сохраняем результаты проверок в логовый файл.

In [19]:
c = 1

for batch_start in range(0, total_rows, batch_size):
    print(f"\n Батч номер {c}:")
    batch = df.limit(batch_start + batch_size).subtract(df.limit(batch_start))
    batch.createOrReplaceTempView("tmp")
    batch.show()
    scan = Scan()
    scan.set_scan_definition_name("CHLEN")
    scan.set_data_source_name("tmp")
    scan.add_configuration_yaml_str(
    f"""
    soda_cloud:
      host: cloud.soda.io
      api_key_id: {API_KEY_ID}
      api_key_secret: {API_KEY_SECRET}
    """
    )
    scan.add_spark_session(spark, data_source_name="tmp")
    scan.add_sodacl_yaml_file("soda/checks.yaml")

    result_code = scan.execute()
    logs = scan.get_logs_text()
    
    with open("logs/soda_logs.log", "a") as log_file:
        log_file.write(f"\n Итерация {c}:")
        log_file.write("\n")
        log_file.write(logs)
        log_file.write("\n")
        
    c+=1


 Батч номер 1:
+----------+--------------+------------+--------------------+------------------+-----------------------+------------------+--------------------+------------------+-----------------+------------------+--------------------+----------------------+---------------------+---------------------+---------------------+-------------------+-----------------------+
|      Date|     Ship_Type|  Route_Type|         Engine_Type|Maintenance_Status|Speed_Over_Ground_knots|   Engine_Power_kW|Distance_Traveled_nm|      Draft_meters|Weather_Condition| Cargo_Weight_tons|Operational_Cost_USD|Revenue_per_Voyage_USD|Turnaround_Time_hours|Efficiency_nm_per_kWh|Seasonal_Impact_Score|Weekly_Voyage_Count|Average_Load_Percentage|
+----------+--------------+------------+--------------------+------------------+-----------------------+------------------+--------------------+------------------+-----------------+------------------+--------------------+----------------------+---------------------+--------